# 修饰符

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#函数是也是一种对象" data-toc-modified-id="函数是也是一种对象-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>函数是也是一种对象</a></span></li><li><span><a href="#修饰符" data-toc-modified-id="修饰符-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>修饰符</a></span></li><li><span><a href="#用-@-来使用修饰符" data-toc-modified-id="用-@-来使用修饰符-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>用 @ 来使用修饰符</a></span></li><li><span><a href="#例子" data-toc-modified-id="例子-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>例子</a></span></li><li><span><a href="#修饰器工厂" data-toc-modified-id="修饰器工厂-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>修饰器工厂</a></span></li></ul></div>

## 函数是也是一种对象

In [1]:
def foo(x):
    print x
print type(foo)

<type 'function'>


In [2]:
# 查看函数对象拥有的方法
dir(foo)

['__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__hash__',
 '__init__',
 '__module__',
 '__name__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'func_closure',
 'func_code',
 'func_defaults',
 'func_dict',
 'func_doc',
 'func_globals',
 'func_name']

In [4]:
foo.__call__(42)

42


In [5]:
# 等价于
foo(42)

42


In [6]:
# 函数也可以作为参数传入另一个函数
def bar(f,x):
    x += 1
    f(x)
bar(foo,4)

5


## 修饰符

**接受一个函数作为输入，通常输出也是一个函数**

In [9]:
def dec(f):
    print 'I am decorating function', id(f)
    return f

In [10]:
# 将 len 函数作为参数传入这个修饰符函数：
declen = dec(len)

I am decorating function 7119856


In [11]:
# 使用新生函数,功能就和返回的函数一样
declen([10,20,30])

3

In [12]:
def loud(f):
    def new_func(*args, **kw):
        print 'calling with', args, kw
        rtn = f(*args, **kw)
        print 'return value is', rtn
        return rtn
    return new_func

In [13]:
loudlen = loud(len)

In [17]:
loudlen({'a':1,'b':2})

calling with ({'a': 1, 'b': 2},) {}
return value is 2


2

In [18]:
loudlen([10, 20, 30])

calling with ([10, 20, 30],) {}
return value is 3


3

## 用 @ 来使用修饰符

In [19]:
def foo(x):
    print x
    
foo = dec(foo)

I am decorating function 79952816


In [20]:
@dec
def foo(x):
    print x

I am decorating function 79952368


事实上，如果修饰符返回的是一个函数，那么可以链式的使用修饰符：
```python
@dec1
@dec2
def foo(x):
    print x
```
使用修饰符 loud 来定义这个函数：

In [21]:
@loud
def foo(x):
    print x

In [22]:
foo(42)

calling with (42,) {}
42
return value is None


## 例子

In [23]:
# 定义两个修饰器函数：
# 一个将原来的函数值加一，另一个乘二：
def plus_one(f):
    def new_func(x):
        return f(x) + 1
    return new_func
def times_two(f):
    def new_func(x):
        return f(x)*2
    return new_func

In [24]:
@plus_one
@times_two
def foo(x):
    return int(x)

In [25]:
foo(13)

27

## 修饰器工厂

In [26]:
# decorators factories 是返回修饰器的函数，例如：
def super_dec(x, y, z):
    def dec(f):
        def new_func(*args, **kw):
            print x + y + z
            return f(*args, **kw)
        return new_func
    return dec
# 可以把修饰器当参数接收

In [27]:
def super_loud(filename):
    fp = open(filename, 'w')
    def loud(f):
        def new_func(*args, **kw):
            fp.write('calling with' + str(args) + str(kw))
            # 确保内容被写入
            fp.flush()
            fp.close()
            rtn = f(*args, **kw)
            return rtn
        return new_func
    return loud

In [28]:
@super_loud('test.txt')
def foo(x):
    print x

In [29]:
foo(12)

12


In [30]:
with open('test.txt') as fp:
    print fp.read()

calling with(12,){}


In [31]:
import os
os.remove('test.txt')